In [2]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [13]:
import numpy as np
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [6]:
df = pd.read_csv("/content/drive/MyDrive/Data Science and ML for Python 2022 Dortmund/Team assignment T3/hepsiburada.csv")
print(df.info())

target = df['Rating'].values.tolist()
datas = df['Review'].values.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243497 entries, 0 to 243496
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Rating  243497 non-null  int64 
 1   Review  243497 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.7+ MB
None


In [7]:
cutOff = int(len(datas) * 0.80) 
x_train, x_test = datas[:cutOff], datas[cutOff:]
y_train, y_test = target[:cutOff], target[cutOff:]

In [8]:
token_num = 1000

tokenizer = Tokenizer(token_num) 
tokenizer.fit_on_texts(datas)

x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [9]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]

num_tokens = np.array(num_tokens) 
print(f'ortalama yorum uzunlugu: {np.mean(num_tokens)}') 
print(f'en uzun yorum kaç karakter? {np.max(num_tokens)}')
print(f'en uzun yorum kaçıncı indekste bulunuyor? {np.argmax(num_tokens)}')
print("Bu yorum:")
print(x_train[np.argmax(num_tokens)])

max_tokens = int(np.mean(num_tokens) + 2 * np.std(num_tokens))
print(f'max token:{max_tokens}')
"""yorumların %kaçına 0 eklenecek? = %95 """
print(np.sum(num_tokens < max_tokens) / len(num_tokens))

ortalama yorum uzunlugu: 15.472835394275904
en uzun yorum kaç karakter? 224
en uzun yorum kaçıncı indekste bulunuyor? 24555
Bu yorum:
Arkadaşlar telefon 40 saatte teslim edildi satıcı ve kargo hızlı yani.Sadece buradaki yorumlara ve birkaç inceleyici yorumu ile aldım telefonu mağazada vs incelemedim.Gelelim + özelliklere. 1.Ben özellikle mavi renk aldım lakin renk beklediğimden biraz açık çıktı gece mavisi laciverte yakın mavi bekliyordum normal mavi gibi çıktı ama iade edecek kadarda karamsar kalmadım açtım ve kullanmaya başladım.Öncelikle telefonun üzerinde gelen ekran koruyucu bant ile denemeyin o halde çok yavaş çünkü :) 2.Telefonu 1 defa şarj ettim 1 saat 20 dakikada felan doldu ve 32 saattir kullanıyorum bunun 10 saati felan sırf telefonu kurcalamakla vs geçti şarj hala % 46 ki bence harika  3.Telefonun hızı ve parmak izi okuyucu mükemmel hızda ve kesinlikle kasma yok tabi birkaç uygulama haricinde birsey yuklemedık daha  4.Eski telfona mi mover programı yükledim son arananlar li

In [10]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

x_train_pad.shape
x_test_pad.shape
print(f"train shape: {x_train_pad.shape}")
print(f"test shape: {x_test_pad.shape}")

idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(),idx.keys()))

train shape: (194797, 42)
test shape: (48700, 42)


In [11]:
def tokens_to_string(tokens):
    words= [inverse_map[token] for token in tokens if token != 0]
    text = ' '.join(words)
    return text

print(x_train[300])
print(tokens_to_string(x_train_tokens[300]))

Ürünü indirimdeyken aldım hızlı kargo diyordu daha 24 saat dolmadan ürün elimdeydi. Ürün gerçekten başarılı özellikle  bu fiyata kablosuz logitech marka mouse bulmak gerçekten zor. Tavsiye ederim...
ürünü aldım hızlı kargo daha 24 saat ürün elimdeydi ürün gerçekten başarılı özellikle bu fiyata kablosuz marka mouse bulmak gerçekten zor tavsiye ederim


In [14]:
model = Sequential()
embedding_size = 42
model.add(Embedding(input_dim=token_num,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print(model.summary())

model.fit(np.array(x_train_pad), np.array(y_train), epochs= 5, batch_size=256)
tf.keras.models.save_model(model, "hepsiburada.model")
result = model.evaluate(np.array(x_test_pad), np.array(y_test))

print(result[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 42, 42)           42000     
                                                                 
 gru (GRU)                   (None, 42, 16)            2880      
                                                                 
 gru_1 (GRU)                 (None, 42, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 45,677
Trainable params: 45,677
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


None
Epoch 1/5
761/761 [==============================] - 60s 68ms/step - loss: 0.1769 - accuracy: 0.9499
Epoch 2/5
761/761 [==============================] - 50s 65ms/step - loss: 0.1178 - accuracy: 0.9583
Epoch 3/5
761/761 [==============================] - 52s 68ms/step - loss: 0.1086 - accuracy: 0.9611
Epoch 4/5
761/761 [==============================] - 50s 65ms/step - loss: 0.1027 - accuracy: 0.9633
Epoch 5/5
761/761 [==============================] - 50s 65ms/step - loss: 0.0983 - accuracy: 0.9648


INFO:tensorflow:Assets written to: hepsiburada.model/assets


INFO:tensorflow:Assets written to: hepsiburada.model/assets


1522/1522 [==============================] - 14s 9ms/step - loss: 0.1280 - accuracy: 0.9531
0.9531006217002869
